In [1]:
# ---------------------------------------------------------------------------- #
# An implementation of https://arxiv.org/pdf/1512.03385.pdf                    #
# See section 4.2 for the model architecture on CIFAR-10                       #
# Some part of the code was referenced from below                              #
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py   #
# ---------------------------------------------------------------------------- #

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

In [5]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

In [6]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

Files already downloaded and verified


In [7]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [8]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [9]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [10]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [13]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.5659
Epoch [1/80], Step [200/500] Loss: 1.5046
Epoch [1/80], Step [300/500] Loss: 1.3132
Epoch [1/80], Step [400/500] Loss: 1.3145
Epoch [1/80], Step [500/500] Loss: 1.1711
Epoch [2/80], Step [100/500] Loss: 1.0754
Epoch [2/80], Step [200/500] Loss: 1.0581
Epoch [2/80], Step [300/500] Loss: 0.9941
Epoch [2/80], Step [400/500] Loss: 0.9575
Epoch [2/80], Step [500/500] Loss: 0.8777
Epoch [3/80], Step [100/500] Loss: 1.0405
Epoch [3/80], Step [200/500] Loss: 0.9029
Epoch [3/80], Step [300/500] Loss: 1.0815
Epoch [3/80], Step [400/500] Loss: 0.8306
Epoch [3/80], Step [500/500] Loss: 1.0403
Epoch [4/80], Step [100/500] Loss: 0.8388
Epoch [4/80], Step [200/500] Loss: 0.7091
Epoch [4/80], Step [300/500] Loss: 0.6002
Epoch [4/80], Step [400/500] Loss: 0.8681
Epoch [4/80], Step [500/500] Loss: 0.8459
Epoch [5/80], Step [100/500] Loss: 0.6004
Epoch [5/80], Step [200/500] Loss: 0.7296
Epoch [5/80], Step [300/500] Loss: 0.7171
Epoch [5/80], Step [400/500] Loss:

Epoch [39/80], Step [200/500] Loss: 0.1455
Epoch [39/80], Step [300/500] Loss: 0.1932
Epoch [39/80], Step [400/500] Loss: 0.1458
Epoch [39/80], Step [500/500] Loss: 0.2583
Epoch [40/80], Step [100/500] Loss: 0.2978
Epoch [40/80], Step [200/500] Loss: 0.2467
Epoch [40/80], Step [300/500] Loss: 0.2230
Epoch [40/80], Step [400/500] Loss: 0.2774
Epoch [40/80], Step [500/500] Loss: 0.2798
Epoch [41/80], Step [100/500] Loss: 0.3183
Epoch [41/80], Step [200/500] Loss: 0.1459
Epoch [41/80], Step [300/500] Loss: 0.2146
Epoch [41/80], Step [400/500] Loss: 0.2055
Epoch [41/80], Step [500/500] Loss: 0.1417
Epoch [42/80], Step [100/500] Loss: 0.2509
Epoch [42/80], Step [200/500] Loss: 0.2922
Epoch [42/80], Step [300/500] Loss: 0.1611
Epoch [42/80], Step [400/500] Loss: 0.1892
Epoch [42/80], Step [500/500] Loss: 0.2801
Epoch [43/80], Step [100/500] Loss: 0.2643
Epoch [43/80], Step [200/500] Loss: 0.1825
Epoch [43/80], Step [300/500] Loss: 0.1554
Epoch [43/80], Step [400/500] Loss: 0.1255
Epoch [43/8

Epoch [77/80], Step [200/500] Loss: 0.2340
Epoch [77/80], Step [300/500] Loss: 0.2393
Epoch [77/80], Step [400/500] Loss: 0.1871
Epoch [77/80], Step [500/500] Loss: 0.2061
Epoch [78/80], Step [100/500] Loss: 0.1513
Epoch [78/80], Step [200/500] Loss: 0.1625
Epoch [78/80], Step [300/500] Loss: 0.2399
Epoch [78/80], Step [400/500] Loss: 0.1719
Epoch [78/80], Step [500/500] Loss: 0.2915
Epoch [79/80], Step [100/500] Loss: 0.1095
Epoch [79/80], Step [200/500] Loss: 0.2412
Epoch [79/80], Step [300/500] Loss: 0.2367
Epoch [79/80], Step [400/500] Loss: 0.1681
Epoch [79/80], Step [500/500] Loss: 0.1087
Epoch [80/80], Step [100/500] Loss: 0.1168
Epoch [80/80], Step [200/500] Loss: 0.2206
Epoch [80/80], Step [300/500] Loss: 0.2495
Epoch [80/80], Step [400/500] Loss: 0.1409
Epoch [80/80], Step [500/500] Loss: 0.0884


In [14]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 87.99 %


In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')